## Query expansion by llm

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
# chain creation
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnableMap
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [21]:
import os
load_dotenv()

groq_llm = ChatGroq(model="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API"),temperature=0.2)
loader = TextLoader("G:\\extracted\\ML\\RAG\\data_management\\text_files\\حباب سکه چیست.txt",encoding="utf-8")

loader = loader.load()

splitter = RecursiveCharacterTextSplitter(
    separators=[" "],
    chunk_size=600,
    chunk_overlap=50 # specially make it more to increase relevancy 
)
docs = splitter.split_documents(loader)
for d in docs[:2]:
    print("*"*50)
    print(d)

**************************************************
page_content='یکی از مفاهیم مهم برای خرید و فروش در بازار سکه، آشنایی با حباب سکه است. احتمالا اصطلاح حباب زیاد به گوش‌تان خورده است که برای انواع بازارها به کار برده می‌شود. این اصطلاح در واقع به تغییرات قیمت در عرضه و تقاضا و حرکت آن به سمت مثبت یا منفی اشاره دارد. اما حباب سکه پیچیده‌تر از این‌هاست! اگر به سرمایه‌ گذاری در بازار سکه علاقه دارید باید به طور کامل با حباب این بازار آشنایی داشته باشید. به این منظور، لازم است که نحوه محاسبه حباب سکه را بدانید و عوامل موثر بر آن را تشخیص دهید. در این مطلب از ره‌آورد، به طور کامل در مورد حباب سکه مطالعه خواهید کرد.

حباب سکه چیست؟
در ابتدا بیایید ماهیت حباب سکه' metadata={'source': 'G:\\extracted\\ML\\RAG\\data_management\\text_files\\حباب سکه چیست.txt'}
**************************************************
page_content='سکه چیست؟
در ابتدا بیایید ماهیت حباب سکه را به خوبی بررسی کنیم. بدون شک، تمام سرمایه‌گذاران به دنبال آن هستند که دارایی موردنظر خود را با پایین‌ترین قیمت ممکن، حتی پایین‌تر ا

In [8]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

vectorstore = FAISS.from_documents(docs, embedding_model)

retriever = vectorstore.as_retriever(
    search_type="similarity", # ('similarity', 'similarity_score_threshold', 'mmr')
    search_kwargs={"k":3})

In [ ]:
# query enhancement prompt
system_prompt = """
تو یک موتور بهبود پرس‌وجوی جستجو (Query Enhancement Engine) هستی.
 سوال را به چند نسخه متفاوت اما معادل معنایی تبدیل کنی


قوانین:
- معنی سوال اصلی نباید تغییر کند
- از اضافه‌گویی و توضیح خودداری کن
- از شمارش جملات پرهیز کن
- فقط خروجی ساختاریافته بده
- اگر سوال مبهم است، آن را شفاف‌سازی کن (نه اینکه حدس بزنی)
- از کلمات انگلیسی رایج فنی در صورت نیاز استفاده کن

فقط از حروف و نشانه‌های زبان فارسی استفاده کن.
از استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.

سوال کاربر : {query}

"""
template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{query}"),
            ]
        )
template

ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\nتو یک موتور بهبود پرس\u200cوجوی جستجو (Query Enhancement Engine) هستی.\n سوال را به چند نسخه متفاوت اما معادل معنایی تبدیل کنی\n\n\nقوانین:\n- معنی سوال اصلی نباید تغییر کند\n- از اضافه\u200cگویی و توضیح خودداری کن\n- از شمارش جملات پرهیز کن\n- فقط خروجی ساختاریافته بده\n- اگر سوال مبهم است، آن را شفاف\u200cسازی کن (نه اینکه حدس بزنی)\n- از کلمات انگلیسی رایج فنی در صورت نیاز استفاده کن\n\nفقط از حروف و نشانه\u200cهای زبان فارسی استفاده کن.\nاز استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.\n\nسوال کاربر : {query}\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])

In [46]:
chain  =  template | groq_llm | StrOutputParser()
chain

ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\nتو یک موتور بهبود پرس\u200cوجوی جستجو (Query Enhancement Engine) هستی.\n سوال را به چند نسخه متفاوت اما معادل معنایی تبدیل کنی\n\n\nقوانین:\n- معنی سوال اصلی نباید تغییر کند\n- از اضافه\u200cگویی و توضیح خودداری کن\n- از شمارش جملات پرهیز کن\n- فقط خروجی ساختاریافته بده\n- اگر سوال مبهم است، آن را شفاف\u200cسازی کن (نه اینکه حدس بزنی)\n- از کلمات انگلیسی رایج فنی در صورت نیاز استفاده کن\n\nفقط از حروف و نشانه\u200cهای زبان فارسی استفاده کن.\nاز استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.\n\nسوال کاربر : {query}\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])
| ChatGroq(profile={'max_input_toke

In [47]:
response = chain.invoke({ 'query' : "حباب منفی سکه" })
print(response)

سوال اصلی: حباب منفی سکه

سوال 1: حباب منفی موجود در سکه
سوال 2: سکه دارای حباب منفی است یا خیر
سوال 3: آیا حباب منفی در سکه وجود دارد
سوال 4: بررسی وجود حباب منفی در سکه


In [48]:
system_prompt = """
تو یک دستیار فارسی‌زبان مفید هستی.
از اطلاعات داده‌شده برای پاسخ دقیق به سوال استفاده کن.
پاسخ باید روان، طبیعی، و مناسب خواندن توسط انسان باشد.
از استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.


اطلاعات: {context}
سوال کاربر : {question}
"""
answer_template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{question}"),
            ]
        )
answer_template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nتو یک دستیار فارسی\u200cزبان مفید هستی.\nاز اطلاعات داده\u200cشده برای پاسخ دقیق به سوال استفاده کن.\nپاسخ باید روان، طبیعی، و مناسب خواندن توسط انسان باشد.\nاز استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.\n\n\nاطلاعات: {context}\nسوال کاربر : {question}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [38]:
# put it in complete pipeline
answer_chain = create_stuff_documents_chain(groq_llm,answer_template)
answer_template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nتو یک دستیار فارسی\u200cزبان مفید هستی.\nاز اطلاعات داده\u200cشده برای پاسخ دقیق به سوال استفاده کن.\nپاسخ باید روان، طبیعی، و مناسب خواندن توسط انسان باشد.\nاز استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.\n\n\nاطلاعات: {context}\nسوال کاربر : {question}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [67]:
full_chain = (
    RunnableMap(
        question =lambda x: x['question'],
        context = lambda x: retriever.invoke(chain.invoke({'query':x['question']}))
    )
    |answer_chain
)
full_chain

{
  question: RunnableLambda(lambda x: x['question']),
  context: RunnableLambda(lambda x: retriever.invoke(chain.invoke({'query': x['question']})))
}
| RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
    context: RunnableLambda(format_docs)
  }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
  | ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nتو یک دستیار فارسی\u200cزبان مفید هستی.\nاز اطلاعات داده\u200cشده برای پاسخ دقیق به سوال استفاده کن.\nپاسخ باید روان، طبیعی، و مناسب خواندن توسط انسان باشد.\nاز استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.\n\n\nاطلاعات: {context}\nسوال کاربر : {question}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_type

In [71]:
print(chain.invoke({ 'query' : "حباب منفی سکه" }))
print("_"*50)
print(full_chain.invoke({'question':'حباب منفی سکه'}))

سوال‌های معادل:

- حباب منفی سکه چیست؟
- تعریف حباب منفی سکه را بفرمایید.
- حباب منفی سکه را توضیح دهید.
- حباب منفی سکه چیست و چه کاربردی دارد؟
__________________________________________________
حباب منفی سکه زمانی اتفاق می‌افتد که قیمت روز سکه از ارزش ذاتی طلای آن کمتر باشد. در این شرایط، قیمت سکه در بازار پایین‌تر از ارزش واقعی آن است. این موضوع می‌تواند به دلیل عوامل مختلفی مانند کمبود عرضه، افزایش تقاضا، یا سایر عوامل اقتصادی و سیاسی رخ دهد.

در حباب منفی سکه، سرمایه‌گذاران می‌توانند با خرید سکه در قیمت پایین، انتظار دارند که قیمت آن در آینده افزایش یابد و سود کسب کنند. در این شرایط، حباب منفی سکه به عنوان یک فرصت سرمایه‌گذاری در نظر گرفته می‌شود.

عوامل موثر بر حباب منفی سکه:

1. کمبود عرضه: اگر عرضه سکه در بازار کم باشد، قیمت آن افزایش می‌یابد و ممکن است از ارزش ذاتی طلای آن بیشتر شود.
2. افزایش تقاضا: اگر تقاضا برای سکه افزایش یابد، قیمت آن نیز افزایش می‌یابد و ممکن است از ارزش ذاتی طلای آن بیشتر شود.
3. عوامل اقتصادی و سیاسی: تغییرات اقتصادی و سیاسی می‌توانند بر قیمت سکه تأثیر